In [ ]:
!pip install "tensorflow>=1.15,<2.0"
!pip install --upgrade tensorflow-hub

In [ ]:
!pip install opencv-python


In [ ]:
!git clone https://github.com/favyen/uns20.git

In [ ]:
#%cd /content/
! rm -rf data/
! mkdir data/
#!wget https://motchallenge.net/data/MOT17.zip
"""!7z x  MOT17.zip
!mv MOT17 data/mot17/"""

In [ ]:
!wget https://favyen.com/files/uns20-model.zip
!7z x  uns20-model.zip -y

In [ ]:
"""from google.colab import drive
drive.mount('/content/drive')"""
video_file="/data/home/sophie/sophie_2024-10-08/aggression_detection/cross-input/GR77_20200512_111314.mp4"#"/content/drive/MyDrive/CDPQ Shared Folder with Partners/Données/Groupe C - Deschambault/Videos/GR77_20200512_111314.mp4"
!mkdir /data/home/sophie/sophie_2024-10-08/aggression_detection/cross-input/data/test
!mkdir /data/home/sophie/sophie_2024-10-08/aggression_detection/cross-input/data/test/training
! rm -rf /data/home/sophie/sophie_2024-10-08/aggression_detection/cross-input/data/test/training/img1
! rm -rf /data/home/sophie/sophie_2024-10-08/aggression_detection/cross-input/data/test/training/img1/*
!mkdir /data/home/sophie/sophie_2024-10-08/aggression_detection/cross-input/data/test/training/img1
store_folder="/data/home/sophie/sophie_2024-10-08/aggression_detection/cross-input/data/test/training/img1/"


import cv2
vidcap = cv2.VideoCapture(video_file)
success,image = vidcap.read()
count = 0
while success:
  cv2.imwrite(store_folder+"%06d.jpg" % count, image)     # save frame as JPEG file
  success,image = vidcap.read()
  print('Read a new frame: ', success)
  count += 1

#data/test/training/img1/ images and  data/test/training/det/det.txt

In [ ]:
#! mv /content/data/images/ /content/data/training/images/
#!mv /content/data/det.txt /content/data/training/
print(count)
%cd uns20/
#!python scripts/mot2json.py ../data/ test
!python infer.py ../model/model ../data



In [ ]:
!pwd

In [ ]:
"""import cv2
import numpy as np
import torch
from yolox.data.data_augment import preproc
from yolox.utils import postprocess
from yolox.exp import get_exp
from yolox.models import YOLOX

def load_model():
    # Initialize your YOLOX model
    exp = get_exp("yolox_s")  # Use the 'yolox_s' experiment config
    model = YOLOX(exp.num_classes, exp.depth, exp.width)

    # Load the model weights
    model.load_state_dict(torch.load("yolox_s.pth"))  # Provide path to your model weights
    model.eval()

    return model

def process_frame(frame, model, input_size=(640, 640)):
    # Preprocess the frame before passing it into the model
    img, _ = preproc(frame, input_size)
    img = torch.from_numpy(img).unsqueeze(0).float()
    img /= 255.0  # Normalize image

    with torch.no_grad():
        outputs = model(img)  # Run the frame through the model
        outputs = postprocess(outputs, 80, 0.5, 0.4)  # Adjust for your threshold values (conf, IoU)

    return outputs

def save_results_to_txt(frame_id, outputs, txt_file):
    # Write results for this frame to the txt file
    if outputs is not None:
        for output in outputs[0]:  # Each frame has a list of detections
            x1, y1, x2, y2, obj_id, score = output
            line = f"{frame_id} {obj_id} {score} {x1} {y1} {x2} {y2}\n"
            txt_file.write(line)

def main(video_path, output_txt_path):
    # Load model
    model = load_model()

    # Open the video file
    cap = cv2.VideoCapture(video_path)

    # Open the output text file
    with open(output_txt_path, "w") as txt_file:
        frame_id = 0

        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            frame_id += 1

            # Process the frame
            outputs = process_frame(frame, model)

            # Save results to text file
            save_results_to_txt(frame_id, outputs, txt_file)

            # Optional: Display the frame
            # cv2.imshow("Frame", frame)
            # if cv2.waitKey(1) & 0xFF == ord("q"):
            #     break

        cap.release()
        cv2.destroyAllWindows()

    print(f"Detection results saved to {output_txt_path}")

if __name__ == "__main__":
    video_path = "input_video.mp4"  # Path to your input video
    output_txt_path = "output_detections.txt"  # Path to save detections
    main(video_path, output_txt_path)
"""

In [ ]:
"""import json
import subprocess
import sys
### format ur data in the structure
data_path = '../data'
split = 'test'

if split == 'train':
	LABELS = ['02', '04', '05', '09', '10', '11', '13']
elif split == 'test':
	LABELS = ['training']

for label in LABELS:
	detections = []
	with open(data_path + '/{}/{}/det/det.txt'.format(split, label), 'r') as f:
		lines = f.readlines()
	for line in lines:
		parts = line.strip().split(',')
		if len(parts) < 7:
			continue
		frame_idx = int(parts[0])
		track_id = int(parts[1])
		left = int(float(parts[2]))
		top = int(float(parts[3]))
		right = left + int(float(parts[4]))
		bottom = top + int(float(parts[5]))
		score = float(parts[6])
		if score < 0.6:
			continue
		while frame_idx >= len(detections):
			detections.append([])
		detections[frame_idx].append({
			'frame_idx': frame_idx,
			'track_id': track_id,
			'left': left,
			'top': top,
			'right': right,
			'bottom': bottom,
		})

	fname = data_path + '/{}/{}/det/det-filter60.json'.format(split, label)
	with open(fname, 'w') as f:
		json.dump(detections, f)
	subprocess.call([
		'python', 'scripts/filter_small.py',
		fname,
		data_path + '/{}/{}/img1/'.format(split, label),
		fname,
	])
"""

In [1]:
%cd /data/home/sophie/sophie_2024-10-08/aggression_detection/cross-input/uns20
#!python scripts/mot2json.py ../data/ test

!pip install geom==0.2
!pip install protobuf==3.20
!pip install scikit-image

import geom
import model

import json
import numpy
import math
import os
import skimage.io, skimage.transform
import sys
import tensorflow as tf
import time

MODEL_PATH = "/data/home/sophie/sophie_2024-10-08/aggression_detection/cross-input/model/model" #sys.argv[1]
data_path = "/data/home/sophie/sophie_2024-10-08/aggression_detection/cross-input/data" #sys.argv[2]

model.BATCH_SIZE = 1
model.SEQ_LEN = 2

SKIP = 2
MAX_AGE = 10
MODE = 'imsp'

LABELS = [""]#["training"]#['MOT17-{}-SDP'.format(x) for x in ['01']]
DETECTION_PATH = "/data/home/sophie/sophie_2024-10-08/aggression_detection/cross-input/GR77_20200512_111314tracking_result.json{}" #data_path + '/test/{}/det/det-filter60.json'
FRAME_PATH = data_path+"/test/training/img1{}" # data_path + '/test/{}/img1/'
OUT_PATH = "uns20.json{}" #data_path + '/test/{}/det/uns20.json'

ORIG_WIDTH = 1280#1920
ORIG_HEIGHT = 720#1080
DETECTION_SCALE = 1
FRAME_SCALE = 1
CROP_SIZE = 64
HIDDEN_SIZE = 4*64

print('initializing model')
m = model.Model(options={'mode': MODE})
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)

m.saver.restore(session, MODEL_PATH)

def get_frame_fname(frame_idx):
	s = str(frame_idx)
	while len(s) < 6:
		s = '0' + s
	return s + '.jpg'

for label in LABELS:
	detection_path = DETECTION_PATH.format(label)
	print('loading detections from {}'.format(detection_path))
	with open(detection_path, 'r') as f:
		raw_detections = json.load(f)

	# auto-detect im width/height
	for frame_idx, dlist in raw_detections.items():
		#print(frame_idx, dlist)
		'''if not dlist or len(dlist) == 0:
			continue'''
		im = skimage.io.imread('{}/{}'.format(FRAME_PATH.format(label), get_frame_fname(frame_idx)))
		im_bounds = geom.Rectangle(geom.Point(0, 0), geom.Point(im.shape[1]*FRAME_SCALE, im.shape[0]*FRAME_SCALE))
		break

	detections = [None for _ in range(len(raw_detections.keys()))]
	#print("###########", detections)
	#for frame_idx, dlist in enumerate(raw_detections):
	for frame_idx, dlist in raw_detections.items():
		#print(frame_idx, dlist)
  

		'''if not dlist or frame_idx % SKIP != 0:
			continue'''
		detections[int(frame_idx)] = []
		for i, d in dlist.items():
			transformed={}
			#rint(d,"44444444444444444444")
			transformed["left"]=max(0,d[0])
			transformed["top"]=max(d[1],0)
			transformed["right"]=d[2]+d[0]
			transformed["bottom"]=d[3]+d[1]
			detections[int(frame_idx)].append(transformed)
			"""rect = geom.Rectangle(
				geom.Point(d['left']//DETECTION_SCALE, d['top']//DETECTION_SCALE),
				geom.Point(d['right']//DETECTION_SCALE, d['bottom']//DETECTION_SCALE)
			)
			rect = im_bounds.clip_rect(rect)
			if rect.lengths().x < 4 or rect.lengths().y < 4:
				continue
			nd = {
				'left': rect.start.x,
				'top': rect.start.y,
				'right': rect.end.x,
				'bottom': rect.end.y,
				'frame_idx': d['frame_idx'],
			}
			detections[frame_idx].append(nd)"""

	def zip_frame_info(detections, frame_idx):
		im = skimage.io.imread('{}/{}'.format(FRAME_PATH.format(label), get_frame_fname(frame_idx)))
		im_bounds = geom.Rectangle(
			geom.Point(0, 0),
			geom.Point(im.shape[0], im.shape[1]))
		info = []
		for detection in detections:
			rect = geom.Rectangle(
				geom.Point(detection['top']//FRAME_SCALE, detection['left']//FRAME_SCALE),
				geom.Point(detection['bottom']//FRAME_SCALE, detection['right']//FRAME_SCALE)
			)
			crop = im[rect.start.x:rect.end.x, rect.start.y:rect.end.y, :]
			resize_factor = min([float(CROP_SIZE) / crop.shape[0], float(CROP_SIZE) / crop.shape[1]])
			crop = (skimage.transform.resize(crop, [int(crop.shape[0] * resize_factor), int(crop.shape[1] * resize_factor)])*255).astype('uint8')
			fix_crop = numpy.zeros((CROP_SIZE, CROP_SIZE, 3), dtype='uint8')
			fix_crop[0:crop.shape[0], 0:crop.shape[1], :] = crop
			detection['width'] = float(detection['right']-detection['left'])/ORIG_WIDTH
			detection['height'] = float(detection['bottom']-detection['top'])/ORIG_HEIGHT
			info.append((detection, fix_crop))
		return info

	def get_loc(detection):
		cx = (detection['left'] + detection['right']) / 2
		cy = (detection['top'] + detection['bottom']) / 2
		cx = float(cx) / ORIG_WIDTH
		cy = float(cy) / ORIG_HEIGHT
		return cx, cy

	def get_stuff(infos):
		def per_info(info):
			images = []
			boxes = []
			for i, (detection, crop) in enumerate(info):
				images.append(crop)
				cx, cy = get_loc(detection)
				boxes.append([cx, cy, detection['width'], detection['height']])
			detections = [get_loc(detection) for detection, _ in info]
			return images, boxes, detections, len(info)

		all_images = []
		all_boxes = []
		all_detections = []
		all_counts = []
		for info in infos:
			images, boxes, detections, count = per_info(info)
			all_images.extend(images)
			all_boxes.extend(boxes)
			all_detections.append(detections)
			all_counts.append(count)

		return all_images, all_boxes, all_detections, all_counts

	def softmax(X, theta = 1.0, axis = None):
		y = numpy.atleast_2d(X)
		if axis is None:
		    axis = next(j[0] for j in enumerate(y.shape) if j[1] > 1)
		y = y * float(theta)
		y = y - numpy.expand_dims(numpy.max(y, axis = axis), axis)
		y = numpy.exp(y)
		ax_sum = numpy.expand_dims(numpy.sum(y, axis = axis), axis)
		p = y / ax_sum
		if len(X.shape) == 1: p = p.flatten()
		return p

	# list of objects (id, detection_idx in latest frame, prev_hidden, time since last match)
	# note: detection_idx should be len(info)+1 for the terminal vertex
	active_objects = None
	track_counter = 0
	for frame_idx in range(0, len(detections)-SKIP, SKIP):
		if not detections[frame_idx] or not detections[frame_idx+SKIP]:
			active_objects = None
			continue

		print(frame_idx, len(detections))
		info1 = zip_frame_info(detections[frame_idx], frame_idx)
		info2 = zip_frame_info(detections[frame_idx+SKIP], frame_idx+SKIP)

		if len(info1) == 0 or len(info2) == 0:
			active_objects = None
			continue

		images1, boxes1, _, counts1 = get_stuff([info1])
		images2, boxes2, _, counts2 = get_stuff([info2])

		if active_objects is None:
			active_objects = []
			for left_idx in range(len(info1)):
				active_objects.append((
					track_counter,
					left_idx,
					numpy.zeros((HIDDEN_SIZE,), dtype='float32'),
					0,
					[images1[left_idx]],
				))
				detections[frame_idx][left_idx]['track_id'] = track_counter
				track_counter += 1

		'''
		outputs_raw, out_mat, cur_hidden, out_logits, mat_finesp, mat_longim = session.run([m.out_mat_reweight, m.out_mat, m.out_hidden, m.out_logits_finesp, m.out_mat_finesp, m.out_mat_longim], feed_dict=feed_dict)

		# take maximum in outputs_raw along the active indices
		outputs = numpy.zeros((len(active_objects), len(info2)+1), dtype='float32')
		for i, obj in enumerate(active_objects):
			cur_finesp = mat_finesp[active_indices[i], :].max(axis=0)
			cur_longim = mat_longim[active_indices[i], :].max(axis=0)
			outputs[i, :] = cur_finesp + cur_longim
			#outputs[i, 0:len(info2)] = outputs_raw[active_indices[i], 0:len(info2)].max(axis=0)
			#outputs[i, len(info2)] = outputs_raw[active_indices[i], len(info2)].min()
		'''

		if MODE == 'imsp' or MODE == 'finesp' or MODE == 'longim':
			# flatten the active objects since each object may have multiple images
			flat_images = []
			flat_boxes = []
			flat_hidden = []
			active_indices = {}
			for i, obj in enumerate(active_objects):
				active_indices[i] = []
				for j in [1, 2, 4, 8, 16]:
				#for j in range(1, len(obj[4])+1, len(obj[4])//5+1):
					if len(obj[4]) < j:
						continue
					# use image from stored history, but use current box
					active_indices[i].append(len(flat_images))
					flat_images.append(obj[4][-j])
					if obj[1] < len(info1):
						flat_boxes.append(boxes1[obj[1]])
					else:
						flat_boxes.append(numpy.zeros((4,), dtype='float32'))
					flat_hidden.append(obj[2])

			feed_dict = {
				m.raw_images: flat_images + images2,
				m.input_boxes: flat_boxes + boxes2,
				m.n_image: [[len(flat_images), len(images2), 0]],
				m.is_training: False,
				m.infer_sel: range(len(flat_images)),
				m.infer_hidden: flat_hidden,
			}

			longim_logits, finesp_logits, pre_cur_hidden = session.run([m.out_logits_longim, m.out_logits_finesp, m.out_hidden], feed_dict=feed_dict)
			longim_out_logits = numpy.zeros((len(active_objects), len(info2)+1), dtype='float32')
			finesp_out_logits = numpy.zeros((len(active_objects), len(info2)+1), dtype='float32')
			cur_hidden = numpy.zeros((len(active_objects), len(info2)+1, HIDDEN_SIZE), dtype='float32')
			for i, obj in enumerate(active_objects):
				longim_out_logits[i, 0:len(info2)] = longim_logits[active_indices[i], 0:len(info2)].mean(axis=0)
				longim_out_logits[i, len(info2)] = longim_logits[active_indices[i], len(info2)].min()
				finesp_out_logits[i, 0:len(info2)] = finesp_logits[active_indices[i], 0:len(info2)].mean(axis=0)
				finesp_out_logits[i, len(info2)] = finesp_logits[active_indices[i], len(info2)].min()
				cur_hidden[i, :, :] = pre_cur_hidden[active_indices[i][0], :, :]
			#longim_mat = softmax(longim_out_logits, axis=1)
			#finesp_mat = softmax(finesp_out_logits, axis=1)
			longim_mat = numpy.minimum(softmax(longim_out_logits, axis=0), softmax(longim_out_logits, axis=1))
			finesp_mat = numpy.minimum(softmax(finesp_out_logits, axis=0), softmax(finesp_out_logits, axis=1))
			outputs = numpy.minimum(longim_mat, finesp_mat)
			#outputs = numpy.minimum(longim_out_logits, finesp_out_logits)
			#outputs = (longim_out_logits+finesp_out_logits)/2
			if MODE == 'finesp':
				outputs = finesp_mat
			elif MODE == 'longim':
				outputs = longim_mat
		else:
			feed_dict = {
				m.raw_images: images1 + images2,
				m.input_boxes: boxes1 + boxes2,
				m.is_training: False,
				m.infer_sel: [obj[1] for obj in active_objects],
				m.infer_hidden: [obj[2] for obj in active_objects],
			}
			if MODE == 'occl':
				feed_dict[m.a_counts] = [len(images1), len(images2)]
			else:
				feed_dict[m.n_image] = [[len(images1), len(images2), 0]]
			outputs, out_mat, out_logits, cur_hidden = session.run([m.out_mat_reweight, m.out_mat, m.out_logits, m.out_hidden], feed_dict=feed_dict)
			outputs = out_mat

		# vote on best next frame: idx1->(output,idx2)
		votes = {}
		for i in range(len(active_objects)):
			for j in range(len(info2)+1):
				output = outputs[i, j]
				#if j == len(info2) and out_logits[active_indices[i][0], :].argmax() == len(info2):
				#if j == len(info2) and longim_out_logits[:, outputs[i, :].argmax()].argmax() != i:
				#if j == len(info2) and longim_out_logits[i, :].max() < 1:
				if MODE == 'imsp' and j != len(info2) and (longim_out_logits[i, j] < 0 or finesp_out_logits[i, j] < 0):
					output = -100.0
				elif MODE == 'finesp' and j != len(info2) and finesp_out_logits[i, j] < 0:
					output = -100.0
				#if j == len(info2):
				#	output = -2
				if i not in votes or output > votes[i][0]:
					if j < len(info2):
						votes[i] = (output, j)
					else:
						votes[i] = (output, None)
		# group by receiver and vote on max idx2->idx1 to eliminate duplicates
		votes2 = {}
		for idx1, t in votes.items():
			output, idx2 = t
			if idx2 is not None and (idx2 not in votes2 or output > votes2[idx2][0]):
				votes2[idx2] = (output, idx1)
		forward_matches = {idx1: idx2 for (idx2, (_, idx1)) in votes2.items()}

		def get_hidden(idx1, idx2):
			if model.__name__ == 'occl3b_model':
				return cur_hidden[idx1, :]
			else:
				return cur_hidden[idx1, idx2, :]

		new_objects = []
		used_idx2s = set()
		for idx1, obj in enumerate(active_objects):
			if idx1 in forward_matches:
				idx2 = forward_matches[idx1]
				new_objects.append((
					obj[0],
					idx2,
					get_hidden(idx1, idx2),
					#numpy.zeros((64,), dtype='float32'),
					0,
					obj[4] + [images2[idx2]],
				))
				used_idx2s.add(idx2)
				detections[frame_idx+SKIP][idx2]['track_id'] = obj[0]
			elif obj[3] < MAX_AGE:
				idx2 = votes[idx1][1]
				if idx2 is None or True:
					idx2 = len(info2)
				new_objects.append((
					obj[0],
					idx2,
					get_hidden(idx1, idx2),
					#numpy.zeros((64,), dtype='float32'),
					obj[3]+1,
					obj[4],
				))

		for idx2 in range(len(info2)):
			if idx2 in used_idx2s:
				continue
			new_objects.append((
				track_counter,
				idx2,
				numpy.zeros((HIDDEN_SIZE,), dtype='float32'),
				0,
				[images2[idx2]],
			))
			detections[frame_idx+SKIP][idx2]['track_id'] = track_counter
			track_counter += 1

		active_objects = new_objects

	ndetections = [None for _ in detections]
	for frame_idx, dlist in enumerate(detections):
		if not dlist:
			continue
		dlist = [d for d in dlist if 'track_id' in d]
		if not dlist:
			continue
		ndetections[frame_idx] = dlist
	detections = ndetections

	with open(OUT_PATH.format(label), 'w') as f:
		json.dump(detections, f)


/data/home/sophie/sophie_2024-10-08/aggression_detection/cross-input/uns20
initializing model





Instructions for updating:
Use `tf.global_variables_initializer` instead.



INFO:tensorflow:Restoring parameters from /data/home/sophie/sophie_2024-10-08/aggression_detection/cross-input/model/model
loading detections from /data/home/sophie/sophie_2024-10-08/aggression_detection/cross-input/GR77_20200512_111314tracking_result.json


2025-04-28 01:56:01.993789: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2025-04-28 01:56:01.999429: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 3593330000 Hz
2025-04-28 01:56:02.002241: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x62477c848630 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2025-04-28 01:56:02.002259: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version


0 14783
2 14783
4 14783
6 14783
8 14783
10 14783
12 14783
14 14783
16 14783
18 14783
20 14783
22 14783
24 14783
26 14783
28 14783
30 14783
32 14783
34 14783
36 14783
38 14783
40 14783
42 14783
44 14783
46 14783
48 14783
50 14783
52 14783
54 14783
56 14783
58 14783
60 14783
62 14783
64 14783
66 14783
68 14783
70 14783
72 14783
74 14783
76 14783
78 14783
80 14783
82 14783
84 14783
86 14783
88 14783
90 14783
92 14783
94 14783
96 14783
98 14783
100 14783
102 14783
104 14783
106 14783
108 14783
110 14783
112 14783
114 14783
116 14783
118 14783
120 14783
122 14783
124 14783
126 14783
128 14783
130 14783
132 14783
134 14783
136 14783
138 14783
140 14783
142 14783
144 14783
146 14783
148 14783
150 14783
152 14783
154 14783
156 14783
158 14783
160 14783
162 14783
164 14783
166 14783
168 14783
170 14783
172 14783
174 14783
176 14783
178 14783
180 14783
182 14783
184 14783
186 14783
188 14783
190 14783
192 14783
194 14783
196 14783
198 14783
200 14783
202 14783
204 14783
206 14783
208 14783
210 1

In [7]:
len(data)

14783

In [ ]:
import json
import cv2
#video_file="/content/drive/MyDrive/CDPQ Shared Folder with Partners/Données/Groupe C - Deschambault/Videos/GR77_20200512_111314.mp4"
video_file="/data/home/sophie/sophie_2024-10-08/aggression_detection/cross-input/GR77_20200512_111314.mp4"

with open('/data/home/sophie/sophie_2024-10-08/aggression_detection/cross-input/uns20/uns20.json') as f:
  data = json.load(f)
vidcap = cv2.VideoCapture(video_file)
fourcc = cv2.VideoWriter_fourcc('m', 'p', '4', 'v')
video_size= (int(vidcap.get(cv2.CAP_PROP_FRAME_WIDTH)),
                                            int(vidcap.get(cv2.CAP_PROP_FRAME_HEIGHT)))
out = cv2.VideoWriter('newgr77_ssd_out_3.mp4',fourcc, vidcap.get(cv2.CAP_PROP_FPS), video_size)

tracking_result={}
for idx, frame_output in enumerate(data):
  if frame_output!=None:
    dct={}
    frame_id = idx#int(frame_output[0]["frame_idx"])
    vidcap.set(cv2.CAP_PROP_POS_FRAMES,frame_id);
    #Read the next frame from the video. If you set frame 749 above then the code will return the last frame.
    ret, frame = vidcap.read()
    #print(frame)

    print(frame_id)
    for track in frame_output:
      #frame_id = int(track["frame_idx"])

      bbox = [track["left"], track["top"], track["right"],track["bottom"] ]
      id_num = str(track["track_id"]) #Get the ID for the particular track.
      dct[id_num]=(int(bbox[0]), int(bbox[1]), int(bbox[2])- int(bbox[0]), int(bbox[3])-int(bbox[1]) )
      #Draw bbox from tracker.
      cv2.rectangle(frame, (int(bbox[0]), int(bbox[1])), (int(bbox[2]), int(bbox[3])),(255,255,255), 2)
      cv2.putText(frame, str(id_num),(int(bbox[0]), int(bbox[1])),0, 5e-3 * 200, (0,255,0),2)
    #cv2_imshow(frame)


    out.write(frame)
    tracking_result[frame_id]=dct

out.release()
with open(video_file.split('.mp4')[0]+'-cross-input-consistency-tracking_resut.json', 'w') as outfile:
    json.dump(tracking_result, outfile)
print("all finished well")


print(data[:10])

0
2
4
6
8
10
12
14
16
18
20
22
24
26
28
30
32
34
36
38
40
42
44
46
48
50
52
54
56
58
60
62
64
66
68
70
72
74
76
78
80
82
84
86
88
90
92
94
96
98
100
102
104
106
108
110
112
114
116
118
120
122
124
126
128
130
132
134
136
138
140
142
144
146
148
150
152
154
156
158
160
162
164
166
168
170
172
174
176
178
180
182
184
186
188
190
192
194
196
198
200
202
204
206
208
210
212
214
216
218
220
222
224
226
228
230
232
234
236
238
240
242
244
246
248
250
252
254
256
258
260
262
264
266
268
270
272
274
276
278
280
282
284
286
288
290
292
294
296
298
300
302
304
306
308
310
312
314
316
318
320
322
324
326
328
330
332
334
336
338
340
342
344
346
348
350
352
354
356
358
360
362
364
366
368
370
372
374
376
378
380
382
384
386
388
390
392
394
396
398
400
402
404
406
408
410
412
414
416
418
420
422
424
426
428
430
432
434
436
438
440
442
444
446
448
450
452
454
456
458
460
462
464
466
468
470
472
474
476
478
480
482
484
486
488
490
492
494
496
498
500
502
504
506
508
510
512
514
516
518
520
522
524
526

In [ ]:
import json
import cv2
from google.colab.patches import cv2_imshow
#video_file="/content/drive/MyDrive/CDPQ Shared Folder with Partners/Données/Groupe C - Deschambault/Videos/GR77_20200512_111314.mp4"
video_file="/content/GR77_20200512_111314.mp4"

with open('/content/uns20.json') as f:
  data = json.load(f)
vidcap = cv2.VideoCapture(video_file)
fourcc = cv2.VideoWriter_fourcc('m', 'p', '4', 'v')
video_size= (int(vidcap.get(cv2.CAP_PROP_FRAME_WIDTH)),
                                            int(vidcap.get(cv2.CAP_PROP_FRAME_HEIGHT)))
out = cv2.VideoWriter('/content/newgr77_ssd_out_3.mp4',fourcc, vidcap.get(cv2.CAP_PROP_FPS), video_size)

tracking_result={}
for frame_output in data:
  if frame_output!=None:
    dct={}
    frame_id = int(frame_output[0]["frame_idx"])
    vidcap.set(cv2.CAP_PROP_POS_FRAMES,frame_id);
    #Read the next frame from the video. If you set frame 749 above then the code will return the last frame.
    ret, frame = vidcap.read()
    #print(frame)

    print(frame_id)
    for track in frame_output:
      #frame_id = int(track["frame_idx"])

      bbox = [track["left"], track["top"], track["right"],track["bottom"] ]
      id_num = str(track["track_id"]) #Get the ID for the particular track.
      dct[id_num]=(int(bbox[0]), int(bbox[1]), int(bbox[2])- int(bbox[0]), int(bbox[3])-int(bbox[1]) )
      #Draw bbox from tracker.
      cv2.rectangle(frame, (int(bbox[0]), int(bbox[1])), (int(bbox[2]), int(bbox[3])),(255,255,255), 2)
      cv2.putText(frame, str(id_num),(int(bbox[0]), int(bbox[1])),0, 5e-3 * 200, (0,255,0),2)
    #cv2_imshow(frame)


    out.write(frame)
    out.write(frame)
    tracking_result[frame_id]=dct

out.release()
with open(video_file.split('.mp4')[0]+'tracking_resut.json', 'w') as outfile:
    json.dump(tracking_result, outfile)
print("all finished well")


print(data[:10])